# 유기동물 정보 가져와서 DB에 넣기

동물보호관리시스템 유기동물 조회 서비스:
<https://www.data.go.kr/dataset/15001096/openapi.do>

공공데이터포털 API를 사용하기 위해 환경변수를 설정합니다.

반드시 Key, Token 등의 내용을 바꿔서 입력해 주세요.

```bash
# Mac, Linux 또는 Windows Git Bash 등에선 다음과 같이 export를 사용하시면 됩니다.
export ANIMAL_API_KEY=abcdefghijklm%2FvSfZeu4FoGpl2eGP%2FzlR5eGh%2F9M8jbOv4HPFZI%2BM7ckZstS8OzkMKHNBkpAtdZX6T8VoRl4Q%3D%3D
# Windows 기본 환경에선 다음과 같이 set을 사용하세요.
set ANIMAL_API_KEY=abcdefghijklm%2FvSfZeu4FoGpl2eGP%2FzlR5eGh%2F9M8jbOv4HPFZI%2BM7ckZstS8OzkMKHNBkpAtdZX6T8VoRl4Q%3D%3D

# Jupyter Notebook 실행
jupyter notebook
```

환경 변수에서 API 키를 가져옵니다.

In [1]:
# 환경 변수에서 API 키 가져오기

import os
from urllib.parse import unquote

API_KEY = unquote(os.environ['ANIMAL_API_KEY'])

## 라이브러리 사용

In [2]:
# requests 사용

import requests

requests.__version__

'2.22.0'

In [3]:
# sqlite3 사용

import sqlite3

sqlite3.version

'2.6.0'

## 동물보호관리시스템 유기동물 조회 서비스 API 사용

DB에 꽉 채워넣읍시다.

In [4]:
# 동물보호관리시스템 유기동물 조회 서비스 API

# 물음표(?) 앞부분
url = 'http://openapi.animal.go.kr/openapi/service/rest/abandonmentPublicSrvc/abandonmentPublic'

# 물음표(?) 뒷부분
payload = {
    'serviceKey': API_KEY,
    'bgnde': '20190801',
    'endde': '20191031',
    'numOfRows': 100_000,
}

# API 호출
response = requests.get(url, params=payload)

print(response)

<Response [200]>


In [5]:
# Response의 Text 길이를 확인합니다.
# 만약 이 값이 너무 작다면 오류가 발생했는지 확인해 보세요.

len(response.text)

23808539

In [6]:
# XML 데이터 파싱

import xml.etree.ElementTree as ET

root = ET.fromstring(response.text)

root

<Element 'response' at 0x102ae6830>

In [7]:
# 컬럼 목록 얻기

item = next(root.iter('item'))

columns = []

for child in item:
    columns.append(child.tag)

columns

['age',
 'careAddr',
 'careNm',
 'careTel',
 'chargeNm',
 'colorCd',
 'desertionNo',
 'filename',
 'happenDt',
 'happenPlace',
 'kindCd',
 'neuterYn',
 'noticeEdt',
 'noticeNo',
 'noticeSdt',
 'officetel',
 'orgNm',
 'popfile',
 'processState',
 'sexCd',
 'specialMark',
 'weight']

## 사용할 컬럼 선택

어떤 컬럼을 사용할지 선택해서 테이블을 만듭시다.

저는 `age`, `colorCd`, `happenDt`, `kindCd`, `orgNm`, `sexCd`, `weight`만 사용하겠습니다.

In [8]:
columns = ['age', 'colorCd', 'happenDt', 'kindCd', 'orgNm', 'sexCd', 'weight']

## Identifier (고유 식별자)

각 동물 데이터에 대한 Identifier를 만들면 개별적으로 관리하기 좋습니다.

학급에서 학생들에게 번호를 지정하면 같은 이름을 가진 학생을 구분하기도 좋고, 훨씬 쉽고 빠르게 데이터에 접근할 수 있습니다.

“아샬”이란 학생의 국어 점수를 찾으려면 꽤 어려울 수 있지만,
(학생 정보가 번호 순서대로 써있다는 전제로) 32번 학생의 국어 점수는 비교적 쉽게 찾을 수 있죠.

SQLite는 `PRIMARY_KEY`와 `AUTOINCREMENT`를 통해 Identifier를 제공합니다.

- [CREATE TABLE](https://www.sqlite.org/lang_createtable.html)
- [SQLite Primary Key](https://www.sqlitetutorial.net/sqlite-primary-key/)
- [SQLite AUTOINCREMENT](https://www.sqlitetutorial.net/sqlite-autoincrement/)

## DB 접속

In [9]:
# 파일로 기록되는 데이터베이스에 연결/접속

connection = sqlite3.connect('test.db')

In [10]:
# 커서 얻기

cursor = connection.cursor()

## 테이블 생성

In [11]:
sql = '''
CREATE TABLE animals (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    age TEXT,
    colorCd TEXT,
    happenDt TEXT,
    kindCd TEXT,
    orgNm TEXT,
    sexCd TEXT,
    weight TEXT
)
'''

cursor.execute(sql)

## XML 파싱

In [12]:
# XML 데이터를 rows에 담기

# rows 준비
rows = []

for item in root.iter('item'):
    # row 준비
    row = {}

    # 원하는 컬럼만 얻습니다.
    for column in columns:
        row[column] = item.find(column).text

    # rows에 추가
    rows.append(row)

In [13]:
# rows 갯수 확인

len(rows)

30330

In [14]:
# 데이터 확인

rows[0]

{'age': '2014(년생)',
 'colorCd': '흰색',
 'happenDt': '20191013',
 'kindCd': '[개] 믹스견',
 'orgNm': '전라남도 곡성군',
 'sexCd': 'M',
 'weight': '20(Kg)'}

## DB에 데이터 넣기

Identifier로 쓰이는 `id`를 제외한 나머지 값을 `INSERT`해야 합니다.

`INSERT INTO [테이블 이름] ([필드 목록]) VALUES ([값 목록])`

필드 목록을 만들기 위해 우리는 `join`을 사용할 겁니다.

- [str.join](https://github.com/ahastudio/til/blob/master/python/str-join.md)
- [SQLite Python: Inserting Data](https://www.sqlitetutorial.net/sqlite-python/insert/)

In [15]:
# 데이터의 키를 모아서 쉼표로 이어줍니다.

', '.join(rows[0].keys())

'age, colorCd, happenDt, kindCd, orgNm, sexCd, weight'

In [16]:
# 필드 목록에 fields란 이름을 붙입니다.

fields = ', '.join(rows[0].keys())

# f-string을 사용하면 중괄호({}) 안에 변수 등을 넣어서 아주 간단히 SQL을 만들 수 있습니다.

sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?)'

print(sql)

INSERT INTO animals (age, colorCd, happenDt, kindCd, orgNm, sexCd, weight) VALUES (?, ?, ?, ?, ?, ?, ?)


In [17]:
# 테이블에 데이터 넣기

for row in rows:
    fields = ', '.join(row.keys())
    sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?)'
    cursor.execute(sql, list(row.values()))

connection.commit()

## DB에 데이터가 잘 들어갔는지 간단히 확인

In [18]:
# 테이블에서 데이터 얻기

cursor.execute('SELECT * FROM animals')

rows = cursor.fetchall()

In [19]:
# 전부 출력하면 무서울테니 갯수만 확인합니다.

print(len(rows))

30330


In [20]:
# 데이터 하나만 확인합니다.

print(rows[0])

(1, '2014(년생)', '흰색', '20191013', '[개] 믹스견', '전라남도 곡성군', 'M', '20(Kg)')


## DB 사용 종료

DB를 사용 후엔 `close`를 하는 게 좋습니다.

In [21]:
connection.close()